In [ ]:
from fastai.vision.all import *

In [ ]:
train_dir = '../input/petfinder-pawpularity-score/train/'
train_df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
train_df['path'] = train_df['Id'].apply(lambda x: train_dir + x + '.jpg')
train_df.tail()

In [ ]:
dblock = DataBlock(
    blocks = (ImageBlock, RegressionBlock),
    get_x = ColReader('path'),
    get_y = ColReader('Pawpularity'),
    splitter = RandomSplitter(0.2),
    item_tfms = Resize(224)
)
dls = dblock.dataloaders(train_df.query('Pawpularity < 100'), bs = 64, num_workers = 4)
dls.show_batch()

In [ ]:
model_path = '../input/petfinder-models/baseline_resnet18.pth'

learn = cnn_learner(dls, resnet18, y_range = (1, 100), metrics = rmse, pretrained = False)
model = learn.model
model.load_state_dict(torch.load(model_path))

In [ ]:
test_dir = '../input/petfinder-pawpularity-score/test/'
test_df = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test_df['path'] = test_df['Id'].apply(lambda x: test_dir + x + '.jpg')
test_df.tail()

In [ ]:
test_dl = dls.test_dl(test_df)

In [ ]:
preds, _ = learn.get_preds(dl = test_dl)

In [ ]:
submission = test_df[['Id']].copy()
submission['Pawpularity'] = preds
submission.to_csv('submission.csv', index = False)
submission